<a href="https://colab.research.google.com/github/BrahiamAcosta/Competencia-Kaggle-Modelos1/blob/main/04%20-%20modelo%20imputando%20datos%20por%20la%20moda%20y%20CatboostClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Cargar los datos

In [1]:
import pandas as pd
import os

os.environ["KAGGLE_CONFIG_DIR"] = "."


!kaggle competitions download -c udea-ai4eng-20242
!unzip udea-ai4eng-20242.zip
!rm udea-ai4eng-20242.zip

 25% 5.00M/20.1M [00:00<00:00, 47.8MB/s]
100% 20.1M/20.1M [00:00<00:00, 122MB/s] 
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [2]:
dtr = pd.read_csv('train.csv')
dtr['RENDIMIENTO_GLOBAL'] = dtr['RENDIMIENTO_GLOBAL'].map({"bajo":1, "medio-bajo":2, "medio-alto":3, "alto":4})
del(dtr['ID'])
dtr.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,3
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,1
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,1
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,4
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,2


In [3]:
#Cargarmos las librerias que vamos a utilizar
!pip install category_encoders
!pip install catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


##Preprocesado de los datos

En este notebook evaluaremos el desempeño de nuestro modelo basado en 2 estrategias distintas de procesamiento de datos:
 <li><b>Pasar los datos crudos a CatboostEncoder: CatboostEncoder </b>es un preprocesador que convierte las variables categóricas en valores numéricos antes de pasarlas al modelo. Con este encoder, no nos preocupamos por imputar los valores nulos, ya que el mismo calculará el peso que tienen estos sobre la variable objetivo
 <li><b>Pasar los datos crudos a el modelo predictivo directamente: </b>Para esta solución, usaremos la plantilla <b>CatboostClassifier</b>. Esta tiene internamente implementaciones para extraer la mayor cantidad de información basado en las columnas con valores categóricos. Al hacerlo de esta forma lo que se busca es intentar aprovechar al máximo las ventajas de CatboostClassifier, para que lleve el proceso de predicción tal y como se planteó desde su creación
</ol>


<b>Nota: </b> Se validará el desempeño de todas las estrategias de procesamiento de datos con la misma plantilla de modelo

##Primer caso de preprocesado


In [110]:
def catboost_encoder(df):
  from category_encoders import CatBoostEncoder
  Xtrcb = df.drop('RENDIMIENTO_GLOBAL', axis=1).astype(str)
  ytrcb = df['RENDIMIENTO_GLOBAL']
  cb_encoder = CatBoostEncoder()
  cb_encoder.fit(Xtrcb, ytrcb)
  return cb_encoder

def transform_data_cb(df):
  encoder = catboost_encoder(df)
  df_encoded = encoder.transform(df.drop('RENDIMIENTO_GLOBAL', axis=1).astype(str))
  df_final = pd.concat([df_encoded,df['RENDIMIENTO_GLOBAL']],axis=1)
  return df_final

###Función de encapsulamiento del proceso con catboost encoder y resultado final de dataframe

In [111]:
def cb_encoder_process(df):
  df = transform_data_cb(df)
  return df

tr2_df = cb_encoder_process(dtr.copy())
tr2_df.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,2.394582,2.313639,2.557381,2.947611,2.459211,2.628079,2.568765,2.497429,3.145015,2.69944,3
1,2.394582,2.629458,2.454779,2.354643,2.870430,2.628079,2.040739,2.658875,2.504667,2.69944,1
2,2.584067,2.479804,2.557381,2.354643,2.422447,2.628079,2.568765,2.508422,2.490187,2.69944,1
3,2.458676,2.298817,2.734278,2.710806,2.870430,2.982042,2.568765,2.703821,2.490187,2.69944,4
4,2.394582,2.466305,2.667598,2.354643,2.435168,2.628079,2.568765,2.239256,2.198277,2.69944,2


##Segundo caso de preprocesado (Pasar los datos crudos al casificador) y validar el rendimiento de las estrategias en el modelo:

###Plantilla de modelo base (sin optimizar):

La plantilla que se usará para validar la mejor estratega de pre procesamiento será un CatboostClassifier con iteraciones máximas a 500.
Luego se optimizará el modelo resultante de la mejor estrategía de preprocesamiento

In [82]:
#Dataframe crudo
tr3_df = dtr.copy()

In [112]:
#dataframes imputados con encoders:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
strategies = {
    'cb_encoder': tr2_df

}

results = {}

for strategy_name, strategy_df in strategies.items():
    print(f"Procesando estrategia: {strategy_name}")

    X = strategy_df.drop('RENDIMIENTO_GLOBAL', axis=1)
    y = strategy_df['RENDIMIENTO_GLOBAL']

    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

    model = CatBoostClassifier(
        iterations=500,
        task_type="GPU",
        devices="0",
        verbose=0  # Evitar salida detallada
    )

    cv_scores = cross_val_score(
        model,
        X,
        y,
        cv=3,
        scoring='accuracy'
    )

    model.fit(Xtr, ytr)

    train_accuracy = model.score(Xtr, ytr)
    test_accuracy = model.score(Xte, yte)

    # Guardar resultados
    results[strategy_name] = {
        "train_accuracy": train_accuracy,
        "test_accuracy": test_accuracy,
        "cv_mean_accuracy": np.mean(cv_scores),
        "cv_std_accuracy": np.std(cv_scores),
        "cv_scores": cv_scores.tolist()
    }


Procesando estrategia: cb_encoder


In [113]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

# Calibración con datos crudos
X = tr3_df.drop('RENDIMIENTO_GLOBAL', axis=1).astype(str)
y = tr3_df['RENDIMIENTO_GLOBAL']

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar las características categóricas
categorical_features_indices = [X.columns.get_loc(col) for col in X.select_dtypes(include=['object']).columns]

model = CatBoostClassifier(
    iterations=500,
    task_type="GPU",
    devices="0",
    verbose=20 #Salida cada 20 iteraciones
)

# Validación cruzada
cv_scores = cross_val_score(
    model,
    X,
    y,
    cv=3,
    scoring='accuracy',
    fit_params={'cat_features': categorical_features_indices}  # Especificar las columnas categóricas
)

model.fit(Xtr, ytr, cat_features=categorical_features_indices)
train_score = model.score(Xtr, ytr)
test_score = model.score(Xte, yte)

# Guardar los resultados
results['raw data'] = {
    "train_score": train_score,
    "test_score": test_score,
    "mean_accuracy": np.mean(cv_scores),
    "std_accuracy": np.std(cv_scores),
    "fold_scores": cv_scores.tolist()
}

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


Learning rate set to 0.40417
0:	learn: 1.3266025	total: 25.5ms	remaining: 12.7s
20:	learn: 1.2098712	total: 496ms	remaining: 11.3s
40:	learn: 1.2012407	total: 853ms	remaining: 9.55s
60:	learn: 1.1971363	total: 1.17s	remaining: 8.41s
80:	learn: 1.1938768	total: 1.49s	remaining: 7.73s
100:	learn: 1.1916143	total: 1.84s	remaining: 7.26s
120:	learn: 1.1890588	total: 2.17s	remaining: 6.8s
140:	learn: 1.1873097	total: 2.49s	remaining: 6.34s
160:	learn: 1.1856984	total: 2.84s	remaining: 5.99s
180:	learn: 1.1841407	total: 3.16s	remaining: 5.57s
200:	learn: 1.1824755	total: 3.49s	remaining: 5.19s
220:	learn: 1.1807821	total: 3.85s	remaining: 4.86s
240:	learn: 1.1793109	total: 4.16s	remaining: 4.47s
260:	learn: 1.1778396	total: 4.5s	remaining: 4.12s
280:	learn: 1.1764280	total: 4.85s	remaining: 3.78s
300:	learn: 1.1749331	total: 5.18s	remaining: 3.42s
320:	learn: 1.1735978	total: 5.5s	remaining: 3.07s
340:	learn: 1.1723069	total: 5.86s	remaining: 2.73s
360:	learn: 1.1707218	total: 6.19s	remainin

###Resultados preliminares:

In [114]:
for strategy_name, scores in results.items():
    print(f"{strategy_name}: Train Accuracy = {scores['train_accuracy'] if 'train_accuracy' in scores else scores['train_score']:.4f}, Test Accuracy = {scores['test_accuracy'] if 'test_accuracy' in scores else scores['test_score']:.4f}")
    print("----------")

cb_encoder: Train Accuracy = 0.4624, Test Accuracy = 0.4379
----------
raw data: Train Accuracy = 0.4577, Test Accuracy = 0.4385
----------


Del análisis anterior, vemos como el mejor rendimiento en test fue el obtenido por el clasificador, dejando que maneje de forma nativa las categorías

###Optimizar el modelo

Ahora, para buscar un mejor desempeño, se volverá a entrenar el modelo con la estrategia ganadora, por medio de modificar sus hiperparámetros

In [104]:
#Buscamos los mejores parametros
from catboost import CatBoostClassifier

X = tr3_df.drop('RENDIMIENTO_GLOBAL', axis=1).astype(str)
y = tr3_df['RENDIMIENTO_GLOBAL']


# Identificar las características categóricas
categorical_features_indices = [X.columns.get_loc(col) for col in X.select_dtypes(include=['object']).columns]

best_model = CatBoostClassifier(
    iterations=500,
    task_type="GPU",
    devices="0",
    verbose=20 #Salida cada 20 iteraciones
)

best_model.fit(X, y, cat_features=categorical_features_indices)

Learning rate set to 0.439023
0:	learn: 1.3230613	total: 85.8ms	remaining: 42.8s
20:	learn: 1.2078599	total: 1.56s	remaining: 35.6s
40:	learn: 1.2001522	total: 2.05s	remaining: 23s
60:	learn: 1.1960865	total: 2.44s	remaining: 17.6s
80:	learn: 1.1931606	total: 2.87s	remaining: 14.8s
100:	learn: 1.1911045	total: 3.27s	remaining: 12.9s
120:	learn: 1.1892443	total: 3.67s	remaining: 11.5s
140:	learn: 1.1878963	total: 4.1s	remaining: 10.4s
160:	learn: 1.1863171	total: 4.5s	remaining: 9.46s
180:	learn: 1.1848766	total: 4.94s	remaining: 8.7s
200:	learn: 1.1835718	total: 5.36s	remaining: 7.97s
220:	learn: 1.1823335	total: 5.76s	remaining: 7.28s
240:	learn: 1.1810467	total: 6.2s	remaining: 6.66s
260:	learn: 1.1800519	total: 6.6s	remaining: 6.04s
280:	learn: 1.1789408	total: 7.04s	remaining: 5.49s
300:	learn: 1.1779235	total: 7.44s	remaining: 4.92s
320:	learn: 1.1768991	total: 7.87s	remaining: 4.39s
340:	learn: 1.1756989	total: 8.27s	remaining: 3.86s
360:	learn: 1.1746977	total: 8.68s	remaining: 

##Prediccion del archivo de test

In [105]:
dts = pd.read_csv('test.csv')
Xts = dts.copy()
del(Xts['Unnamed: 0'])
del(Xts['ID'])
Xts.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


In [106]:
preds = best_model.predict(Xts.astype(str))
preds = preds.flatten()
preds

array([1, 3, 4, ..., 3, 4, 4])

In [107]:
preds = pd.Series(preds,name='RENDIMIENTO_GLOBAL')
preds = preds.map({1: "bajo", 2: "medio-bajo", 3: "medio-alto", 4: "alto"})
preds

,RENDIMIENTO_GLOBAL
0,bajo
1,medio-alto
2,alto
3,bajo
4,bajo
...,...
296781,medio-bajo
296782,medio-alto
296783,medio-alto
296784,alto


In [108]:
#csv de prediccion
submission = pd.DataFrame([dts.ID,preds]).T
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


In [109]:
submission.to_csv('submission.csv',index=False)